# Cats vs Dogs

In [1]:
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear  
# inline in the notebook rather than in a new window.
%matplotlib inline

# set default size of plots
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
tfds.list_builders()

['abstract_reasoning',
 'aflw2k3d',
 'amazon_us_reviews',
 'bair_robot_pushing_small',
 'bigearthnet',
 'binarized_mnist',
 'binary_alpha_digits',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_corrupted',
 'clevr',
 'cnn_dailymail',
 'coco',
 'coco2014',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eurosat',
 'fashion_mnist',
 'flores',
 'food101',
 'gap',
 'glue',
 'groove',
 'higgs',
 'horses_or_humans',
 'image_label_folder',
 'imagenet2012',
 'imagenet2012_corrupted',
 'imdb_reviews',
 'iris',
 'kitti',
 'kmnist',
 'lfw',
 'lm1b',
 'lsun',
 'mnist',
 'mnist_corrupted',
 'moving_mnist',
 'multi_nli',
 'nsynth',
 'omniglot',

In [16]:
builder = tfds.builder('cats_vs_dogs')
info = builder.info
print(info)
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds_train = builder.as_dataset(split='train', shuffle_files=True, as_supervised=True)
print(ds_train)

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    version=2.0.1,
    description='A large set of images of cats and dogs.There are 1738 corrupted images that are dropped.',
    urls=['https://www.microsoft.com/en-us/download/details.aspx?id=54765'],
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=23262,
    splits={
        'train': 23262,
    },
    supervised_keys=('image', 'label'),
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
    title = {Asirra: A CAPTCHA that Exploits Interest-Aligned Manual Image Categorization},
    booktitle = {Proceedings of 14th ACM Conference on Computer and Communications Security (CCS)},
    year

In [17]:
info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})

In [18]:
type(info.features['image'])

tensorflow_datasets.core.features.image_feature.Image

In [19]:
print(info.features["label"].num_classes)
print(info.features["label"].names)
print(info.features["label"].int2str(0))  # Human readable version (0 -> 'cat')
print(info.features["label"].int2str(1))  # Human readable version (0 -> 'cat')
print(info.features["label"].str2int('cat'))
print(info.features["label"].str2int('dog'))

2
['cat', 'dog']
cat
dog
0
1


In [20]:
print(info.features.shape)
print(info.features.dtype)
print(info.features['image'].shape)
print(info.features['image'].dtype)

{'image': (None, None, 3), 'image/filename': (), 'label': ()}
{'image': tf.uint8, 'image/filename': tf.string, 'label': tf.int64}
(None, None, 3)
<dtype: 'uint8'>


In [21]:
print(info.splits)
print(list(info.splits.keys()))

{'train': <tfds.core.SplitInfo num_examples=23262>}
['train']


In [22]:
print(info.splits['train'].num_examples)
print(info.splits['train'].num_shards)

23262
20


In [29]:
ds_train, info = tfds.load('cats_vs_dogs', split='train', shuffle_files=True, with_info=True, as_supervised=True)
assert isinstance(ds_train, tf.data.Dataset)

In [30]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [31]:
ds = ds_train.take(10)

for image, label in ds:
    print(image.shape, label)

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [240,320,3] and element 1 had shape [484,500,3]. [Op:IteratorGetNextSync]

# VGG16 Feature Extractors

In [ ]:
# import the necessary packages
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from utils.hdf5_dataset_writer import HDF5DatasetWriter
import numpy as np
import progressbar
import random
import os

from utils.image_preprocessor import ImagePreprocessor
from utils.image_preprocessor import ResizePreprocessor
from utils.image_preprocessor import ImageToArrayPreprocessor

In [ ]:
# load the VGG16 network
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
model.summary()

In [ ]:
# initialize the HDF5 dataset writer, then store the class label names in the dataset
sample_num = 23262
output = './output/feature_persistence/vgg16-cats-vs-dogs-features.hdf5'
buffer_size = 1024
labelNames = ['cat', 'dog']
dataset = HDF5DatasetWriter((sample_num, 512 * 7 * 7), output, dataKey="features", bufSize=buffer_size) 
dataset.storeClassLabels(labelNames)

In [ ]:
# initialize the progress bar
widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=sample_num, widgets=widgets).start()

# loop over the images in patches
for i in np.arange(0, sample_num, batch_size):
    # extract the batch of images and labels, then initialize the
    # list of actual images that will be passed through the network
    # for feature extraction
    batchImages = trainX[i:i + batch_size]
    batchLabels = trainY[i:i + batch_size]
    batchLabels = np.squeeze(batchLabels)
    
    # subtracting the mean RGB pixel intensity from the ImageNet dataset
    batchImages = imagenet_utils.preprocess_input(batchImages)
    
    # pass the images through the network and use the outputs as our actual features
    features = model.predict(batchImages, batch_size=batch_size)
    
    # reshape the features so that each image is represented by
    # a flattened feature vector of the ‘MaxPooling2D‘ outputs
    features = features.reshape((features.shape[0], 512 * 7 * 7))

    # add the features and labels to our HDF5 dataset
    dataset.add(features, batchLabels)
    pbar.update(i)
    
# close the dataset
dataset.close()
pbar.finish()

In [ ]:
import h5py
db = h5py.File(output)
list(db.keys())

# Old

In [ ]:
from keras.datasets import cifar10
((trainX, trainY), (testX, testY)) = cifar10.load_data()

In [ ]:
sample_num = 1000

In [ ]:
trainX = trainX[:sample_num]
trainX.shape

In [ ]:
trainY = trainY[:sample_num]
trainY.shape

In [ ]:
aap = ResizePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()
ip = ImagePreprocessor(preprocessors=[aap, iap])

In [ ]:
trainX = ip.preprocess(trainX)

In [ ]:
plt.imshow(trainX[0])

In [ ]:
# path to output HDF5 file
output = './output/feature_persistence/vgg16-cifar10-features.hdf5'
# batch size of images to be passed through network, default=32
batch_size = 32
# size of feature extraction buffer, default=1024
buffer_size = 1024

In [ ]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# initialize the HDF5 dataset writer, then store the class label names in the dataset
dataset = HDF5DatasetWriter((sample_num, 512 * 7 * 7), output, dataKey="features", bufSize=buffer_size) 
dataset.storeClassLabels(labelNames)

In [ ]:
# initialize the progress bar
widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=sample_num, widgets=widgets).start()

# loop over the images in patches
for i in np.arange(0, sample_num, batch_size):
    # extract the batch of images and labels, then initialize the
    # list of actual images that will be passed through the network
    # for feature extraction
    batchImages = trainX[i:i + batch_size]
    batchLabels = trainY[i:i + batch_size]
    batchLabels = np.squeeze(batchLabels)
    
    # subtracting the mean RGB pixel intensity from the ImageNet dataset
    batchImages = imagenet_utils.preprocess_input(batchImages)
    
    # pass the images through the network and use the outputs as our actual features
    features = model.predict(batchImages, batch_size=batch_size)
    
    # reshape the features so that each image is represented by
    # a flattened feature vector of the ‘MaxPooling2D‘ outputs
    features = features.reshape((features.shape[0], 512 * 7 * 7))

    # add the features and labels to our HDF5 dataset
    dataset.add(features, batchLabels)
    pbar.update(i)
    
# close the dataset
dataset.close()
pbar.finish()

In [ ]:
import h5py
db = h5py.File(output)
list(db.keys())

# Training a Classifier on Extracted Features CIFAR10

In [ ]:
# import the necessary packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import argparse
import pickle
import h5py

In [ ]:
# construct the arguments
# path HDF5 database
output = './output/feature_persistence/vgg16-cifar10-features.hdf5'
# path to output model
model_path = './output/feature_persistence/vgg16-cifar10-model.cpickle'
# num of jobs to run when tuning hyperparameters
jobs = -1

In [ ]:
# open the HDF5 database for reading then determine the index of
# the training and testing split, provided that this data was
# already shuffled *prior* to writing it to disk
db = h5py.File(output, "r")
i = int(db["labels"].shape[0] * 0.75)

# define the set of parameters that we want to tune then start a
# grid search where we evaluate our model for each value of C
print("[INFO] tuning hyperparameters...")
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
model = GridSearchCV(LogisticRegression(), params, cv=3, n_jobs=jobs)
model.fit(db["features"][:i], db["labels"][:i])
print("[INFO] best hyperparameters: {}".format(model.best_params_))

# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(db["features"][i:])
print(classification_report(db["labels"][i:], preds, target_names=db["label_names"]))

In [ ]:
# serialize the model to disk
print("[INFO] saving model...")
f = open(model_path, "wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()

# close the database
db.close()

Well Done